# Reading, manipulating, and writing data with Astropy - 2

TODO:

This is more advanced - see 1.

Masked tables, table operations, pandas


### A brief aside: Tables vs. Pandas DataFrames

The [Pandas](http://pandas.pydata.org/pandas-docs/stable/) package provides a powerful, high-performance table object via the [DataFrame](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html#pandas.DataFrame) class.  Unfortunately, there are a few shortcomings that prevent its use as a generalized table object in astronomy.  The most crucial is lack of support for multidimensional table columns, which is commonly used in standard FITS data products (e.g., from Chandra, or SDSS catalogs). Pandas `DataFrame` functionality is still very complementary to astropy `Table`s, so `Table`s support converting to and from `DataFrame`s. If you wish to learn more about Pandas, there are many resources available on-line.  A good starting point is the main tutorials site at http://pandas.pydata.org/pandas-docs/stable/tutorials.html.

In [1]:
# Imports we'll need throughout the tutorial:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

from astropy.table import Table
from astropy.io import ascii, fits

Masked tables
-------------

In [ ]:
t2 = Table([['x', 'y', 'z'], 
            [1.1, 2.2, 3.3]],
           names=['name', 'value'],
           masked=True)
t2

In [ ]:
t2['value'].mask = [False, True, False]

In [ ]:
print(t2)

In [ ]:
t2['value'].fill_value = -99
print(t2.filled())

High-level table operations
----------------------------

So far we've just worked with one table at a time and viewed that table as a monolithic entity.  Astropy also supports high-level [Table operations](http://astropy.readthedocs.org/en/stable/table/operations.html) that manipulate multiple tables or view one table as a collection of sub-tables (groups).

 Documentation	                                                                         | Description
---------------------------------------------------------------------------------------- |-----------------------------------------
[Grouped operations](http://astropy.readthedocs.org/en/stable/table/operations.html#id2) | Group tables and columns by keys
[Stack vertically](http://astropy.readthedocs.org/en/stable/table/operations.html#id3)   | Concatenate input tables along rows
[Stack horizontally](http://astropy.readthedocs.org/en/stable/table/operations.html#id4) | Concatenate input tables along columns
[Join](http://astropy.readthedocs.org/en/stable/table/operations.html#join)              | Database-style join of two tables

Here we'll just introduce the join operation but go into more detail on the others in the exercises.

In [ ]:
from astropy.table import join

Now recall our original table `t`:

In [ ]:
t

Now say that we now got some additional flux values from a different reference for a different, but overlapping sample of sources:

In [ ]:
t2 = Table()
t2['name'] = ['larry', 'moe', 'groucho']
t2['flux2'] = [1.4, 3.5, 8.6]

Now we can get a master table of flux measurements which are joined matching the values the `name` column.  This includes every row from each of the two tables, which is known as an **`outer`** join.

In [ ]:
t3 = join(t, t2, keys=['name'], join_type='outer')
print(t3)

In [ ]:
np.mean(t3['flux2'])

Alternately we could choose to keep only rows where both tables had a valid measurement using an **`inner`** join:

In [ ]:
join(t, t2, keys=['name'], join_type='inner')

Writing data
------------

In [ ]:
t3.write('test.fits', overwrite=True)

In [ ]:
t3.write('test.vot', format='votable', overwrite=True)
!more test.vot

Reading data
------------

You can read data using the [Table.read()](http://astropy.readthedocs.org/en/stable/api/astropy.table.Table.html#astropy.table.Table.read) method:

In [ ]:
t4 = Table.read('test.fits')
t4

Some formats, such as FITS and HDF5, are automatically identified by file extention while most others will require ``format`` to be explicitly provided. A number of common ascii formats are supported such as IPAC, sextractor, daophot, and CSV. Refer to the documentation for a full listing.  More details about this are also in the final sections of this notebook.

In [ ]:
Table.read?

In [ ]:
!head 2mass.tbl

In [ ]:
t_2mass = Table.read("2mass.tbl", format="ascii.ipac")
t_2mass.show_in_notebook()

## Working with Pandas
Astropy tables includes  `to_pandas()` and `from_pandas()` [methods](http://docs.astropy.org/en/stable/table/pandas.html)  that facilitate conversion to/from [pandas](http://pandas.pydata.org) `DataFrame` objects.  There are a few caveats in making these conversions:
 - Tables with multi-dimensional columns cannot be converted.
 - Masked values are converted to `numpy.nan`. Numerical columns, int or float, are thus converted to ``numpy.float`` while string columns with missing values are converted to object columns with ``numpy.nan`` values to indicate missing or masked data. Therefore, one cannot always round-trip between `Table` and `DataFrame`.

In [ ]:
import pandas as pd
pd_2mass = t_2mass.to_pandas()
pd_2mass

In [ ]:
t_pd = Table.from_pandas(pd_2mass)
t_pd.show_in_notebook()

As an example of why this might be useful, Pandas can import/export Excel files (which Astropy's io does *not* support), and this provides a way to get such data into `astropy` Tables.

In [ ]:
pd_2mass.to_excel("2mass.xls", index=False)
excel_data = Table.from_pandas(pd.read_excel("2mass.xls"))
excel_data.show_in_notebook()

`DataFrame` has a built-in `plot()` method which is handy for quick-n-dirty visualization of its contents.

In [ ]:
%matplotlib inline
pd_2mass.plot.scatter('j_m', 'h_m')

*****
*****

# Exercises


### Read the data

To start with, read in the two data files representing the master source list and observations source list.  The fields for the two tables are respectively documented in:

- [master_sources](http://cxc.harvard.edu/csc/columns/master.html)
- [obs_sources](http://cxc.harvard.edu/csc/columns/persrc.html)

In [ ]:
master_sources = Table.read('cdfs_master_sources.fits')
obs_sources = Table.read('cdfs_obs_sources.fits')

**`master_sources`**

Each distinct X-ray source identified on the sky is represented in the catalog by a single "master source" entry and one or more "source observation" entries, one for each observation in which the source has been detected. The master source entry records the best estimates of the properties of a source, based on the data extracted from the set of observations in which the source has been detected.  The subset of fields in our exercise table file are:

Name | Description
------ | ------------
msid  | Master source ID
name  | Source name in the Chandra catalog
ra  | Source RA (deg)
dec | Source Dec (deg)

**`obs_sources`**

The individual source entries record all of the properties about a detection extracted from a single observation, as well as associated file-based data products, which are observation-specific.  The subset of fields in our exercise table file are:

Name | Description
------ | ------------
obsid | Observation ID
obi | Observation interval
targname | Target name
gti_obs | Observation date
flux_aper_b | Broad band (0.5 - 7 keV) flux (erg/cm2/sec)
src_cnts_aper_b | Broad band source counts
ra_b | Source RA (deg)
dec_b | Source Dec (deg)
livetime | Observation duration (sec)
posid | Position ID
theta | Off-axis angle (arcmin)
msid | Master source ID

### Exploring the data
Do the following to explore the two tables:

- Display the data for each table in IPython notebook using the normal way of showing the value of a variable.
- Get a list of the column names for each table.  *Hint*: use `<TAB>` completion to easily discover all the attributes and methods, e.g. type `master_sources.` and then hit the `<TAB>` key.
- Find the length of each table.
- Find the column datatypes for each table.

Normally one displays a table in IPython notebook by entering the variable name in a cell and pressing `shift-Enter`.  In a terminal session the default method is using something like `print(my_table)`.  In both cases the `Table` object prefers to display only a screenful of data to prevent having a zillion lines of output if the table is huge.  If you really want to see all the data you can use the [Table.pprint](http://astropy.readthedocs.org/en/stable/api/astropy.table.Table.html#astropy.table.Table.pprint) method. If you are using a Jupyter notebook interface, try the `show_in_notebook()` method.

- Display all the rows of the `master_sources` table using its `pprint()` method.
- If you are working in a regular terminal window (not IPython notebook), try the `more()` method as well.

### Modifying tables
For our analysis we don't actually need the `obi` (observation interval) column in the `obs_sources` table.

- Remove the `obi` column from the `obs_sources` table.

The `gti_obs` column name is a bit obscure (GTI is a good time interval, FWIW).

- Rename the `gti_obs` column to `obs_date`.

It would be nice to have a count rate in addition to the source counts.

- Add a new column `src_rate_aper_b` which is the source counts divided by observation duration in sec.

Some of the sources have a negative net flux in the broad band

### Looking at the observation source data
For each source detected in an individual observation (in the `obs_sources` table), let's look at the source flux values.

- Use the matplotlib [`hist()`]( http://matplotlib.org/api/pyplot_api.html?highlight=pyplot.hist#matplotlib.pyplot.hist) function to make a histogram of the source fluxes.  Since the fluxes vary by orders of magnitude,
  use the `numpy.log10` to put the fluxes in log space.

- Also make the same plot but using only sources within 4 arcmin of the center.  *HINT*: use a boolean mask to select values of `theta` that are less than 4.0.

### Join the master_sources and obs_sources tables

The `master_sources` and `obs_sources` tables share a common `msid` column. What we now want is to join the master RA and Dec positions and master source names with the individual observations table.

- Use the [table.join()](http://astropy.readthedocs.org/en/stable/table/operations.html#join) function to make a single table called `sources` that has the master RA, Dec, and name included for each observation source.

*HINT*: the defaults for `keys` and `join_type='inner'`  are correct in this case, so the simplest possible call to `join()` will work!

- *Intermediate*: Is the length of the new `sources` the same as `obs_sources`?  What happened?

- *Advanced*: Make a scatter plot of the RA (x-axis) and Dec (y-axis) difference between the master source position and the observation source position.  You'll need to use `coordinates`!

### Grouped properties of `sources`

Finally, we can look at the variability properties of sources in the CDFS using the [`group_by()`](http://astropy.readthedocs.org/en/stable/table/operations.html#id2) functionality.  

This method makes a new table in which all the sources with identical master ID are next to each other.

- Make a new table `g_sources` which is the `sources` table grouped by the `msid` key using the `group_by()` method.

The `g_sources` table is just a regular table with all the `sources` in a particular order.  The attribute `g_sources.groups` is an object that provides access to the `msid` sub-groups.  You can access the $i^{th}$ group with `g_sources.groups[i]`.

In addition the `g_sources.groups.indices` attribute is an array with the indicies of the group boundaries.

- Using `np.diff()` find the number of repeat observations of each master sources.  *HINT*: use the indices, Luke.
- Print the 50th group and note which columns are the same for all group members and which are different.  Does this make sense?  In these few observations how many different target names were provided by observers?

#### Aggregation

The real power of grouping comes in the ability to create aggregate values for each of the groups, for instance the mean flux for each unique source.  This is done with the [`aggregate()`](http://astropy.readthedocs.org/en/stable/table/operations.html#aggregation) method, which takes a function reference as its input.  This function must take as input an array of values and return a single value.

Aggregate returns a new table that has a length equal to the number of groups.

- Compute the mean of all columns for each unique source (i.e. each group) using `aggregate` and the `np.mean` function.  Call this table `g_sources_mean`.
- Notice that aggregation cannot form a mean for certain columns and these are dropped from the output.  Use the `join()` function to restore the `master_sources` information to `g_sources_mean`.

# Connecting Tables and FITS

While FITS *image* file access is discussed in another tutorial file, here we consider the interaction of these two: Loading a FITS table and interacting with it as an Astropy Table.

In [ ]:
from astropy.io import fits

In [ ]:
ftab = fits.open('table_example.fits')
ftab[1]

This is not an `astropy.io.fits` table, *not* an Astropy Table like in the examples above. In general the Astropy Table interface is substantially easier to work with, so you'll usually want to just convert the fits table into an Astropy table:

In [ ]:
tab = Table(ftab[1].data)
tab

Note that Astropy includes a "unified I/O" architecture that makes it possible to effectively do this even without using `io.fits` directly at all.  This is usually the easiest approach for simple FITS files:

In [ ]:
tab = Table.read('table_example.fits')
tab

And in fact you can do the same to *write* a fits table:

In [ ]:
tab.write('test_write.fits')

# Reading and Writing ASCII Tables

An equally (if not more) common interchange format in Astronomy is simple ASCII files.  `astropy` provides a large set of ASCII file readers that will read a variety of obscure astronomy formats.  While these are actually *implemented* in the `astropy.io.ascii` sub-package, in general it is easier to use the "unified" I/O to read these directly as tables, like in the examples below.

In [ ]:
Table.read('example.dat', format='ascii')

In [ ]:
Table.read('example.csv', format='ascii.csv')

Note that the reader is often smart enough to determine the type of ASCII file even if you don't specify it (i.e. the `'ascii.csv'` in the above example).  For example, an IPAC format file can be read simply by doing:

In [ ]:
Table.read('example.ipac', format='ascii')

With no need to specify `ascii.ipac`.

The same machinery can also be used to *write* ascii files:

In [ ]:
tab.write('ascii_out.dat', format='ascii')

While the above is often sufficient, it's important to know that the base-line format does *not* support quite a few of the "advanced" features, like inclusion of units.  To do that you'll likely want to use the astropy table "enhanced" CSV:

In [ ]:
tab.write('ascii_out.ecsv', format='ascii.ecsv')